<a href="https://colab.research.google.com/github/RandyTunru/TextSummarizationWithTF-IDF/blob/main/NLP_TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

Program ini merupakan sistem extractive Text Summarization, program ini menggunakan beberapa library seperti nltk, math, re(regex) dan pandas. Program ini memanfaatkan tokenization dari NLTK, serta menggunakan Porter Stemmer, Text summarization ini menggunakan TF-IDF sebagai word embeddings nya.

Secara sederhana, program akan membagi artikel menjadi beberapa sentence (yang akan dianggap sebagai dokumen untuk TF-IDF) lalu program akan menghitung nilai TF-IDF dari setiap token, setelah itu program akan menjumlahkan bobot TF-IDF dalam suatu sentence sehingga menjelaskan apakah suatu sentence dianggap penting atau tidak, program akan menghitung rata rata nilai TF-IDF dari setiap sentence untuk mendapatkan threshold, dan akhirnya program akan mengenerate summary berdasarkan sentences yang memiliki nilai TF-IDF diatas dari threshold

Referensi : https://medium.com/@ashins1997/text-summarization-f2542bc6a167

# Data Preparation

## 1. Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Import Modules

In [ ]:
import pandas as pd

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
import math

ps = PorterStemmer()

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re

## 3. Setting up csv and dataframe

In [ ]:
#Creating a dataframe from the CSV
df = pd.read_csv('/content/drive/MyDrive/csv_coll/CNN_Articels_clean/CNN_Articels_clean.csv')

In [ ]:
df

,Index,Author,Date published,Category,Section,Url,Headline,Description,Keywords,Second headline,Article text
0,0,"Jacopo Prisco, CNN",2021-07-15 02:46:59,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d..."
1,2,"Stephanie Bailey, CNN",2021-05-12 07:52:09,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...
2,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16 02:51:30,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut..."
3,4,"Paul R. La Monica, CNN Business",2022-03-15 09:57:36,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor..."
4,7,Reuters,2022-03-15 11:27:02,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...
...,...,...,...,...,...,...,...,...,...,...,...
4071,4725,Story by Reuters,2021-12-01 10:01:29,sport,tennis,https://www.cnn.com/2021/12/01/tennis/australi...,Australian Open: Australia's vaccine mandate i...,The Australian Open's Covid-19 vaccine mandate...,"tennis, Australian Open: Australia's vaccine m...",Australia's vaccine mandate is not to 'blackma...,(Reuters)The Australian Open's Covid-19 vacci...
4072,4726,"Homero De la Fuente, CNN",2021-12-01 17:56:15,sport,golf,https://www.cnn.com/2021/12/01/golf/south-afri...,Four golfers test positive ahead of South Afri...,Four golfers scheduled to play in the South Af...,"golf, Four golfers test positive ahead of Sout...",Four golfers test positive ahead of South Afri...,(CNN)Four golfers scheduled to play in the So...
4073,4727,"Amy Woodyatt, CNN",2021-12-01 11:32:15,sport,tennis,https://www.cnn.com/2021/12/01/tennis/peng-shu...,Peng Shuai: 'Unanimous conclusion' that tennis...,Long-time International Olympic Committee (IOC...,"tennis, Peng Shuai: 'Unanimous conclusion' tha...",'Unanimous conclusion' that Peng Shuai is 'fin...,(CNN)Long-time International Olympic Committe...
4074,4728,Story by Reuters,2021-12-01 17:27:34,news,europe,https://www.cnn.com/2021/12/01/europe/cow-dung...,"This company is ""zapping"" cow dung with lightn...",A Norwegian technology company has found a way...,"europe, This company is ""zapping"" cow dung wit...",Zapping cow dung with lightning is helping to ...,A Norwegian technology company has found a way...


## 4. Copying Article Text series from dataframe to be processed

In [ ]:
#Creating an array that only contains column 'Article Text" from the dataframe
articleText = df['Article text'].copy(deep=True)

In [ ]:
articleText

0        (CNN)Right now, there's a shortage of truck d...
1        (CNN)Working in a factory or warehouse can me...
2        (CNN)In a Hong Kong warehouse, a swarm of aut...
3       New York (CNN Business)For many years, the wor...
4       The European Union formally approved on Tuesda...
                              ...                        
4071     (Reuters)The Australian Open's Covid-19 vacci...
4072     (CNN)Four golfers scheduled to play in the So...
4073     (CNN)Long-time International Olympic Committe...
4074    A Norwegian technology company has found a way...
4075    This story was excerpted from the November 23 ...
Name: Article text, Length: 4076, dtype: object

In [ ]:
articleText[0]

' (CNN)Right now, there\'s a shortage of truck drivers in the US and worldwide, exacerbated by the e-commerce boom brought on by the pandemic. One solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one. Among them is San Diego-based TuSimple.Founded in 2015, TuSimple has completed about 2 million miles of road tests with its 70 prototype trucks across the US, China and Europe. Although these are simply commercially available trucks retrofitted with its technology, TuSimple has deals in place with two of the world\'s largest truck manufacturers -- Navistar in the US and Traton, Volkswagen\'s trucking business, in Europe -- to design and build fully autonomous models, which it hopes to launch by 2024. Photos: The Yara Birkeland is what its builders call the world\'s first zero-emission, autonomous cargo ship. The ship is scheduled to make its first journey between two Norwegian towns before the end of the year. Click through to see

# Data Cleaning

##Lower Case

In [ ]:
#Modifying articles to all lowercase
for a in range(len(articleText)) :
  articleText[a] = articleText[a].lower()

In [ ]:
articleText

0        (cnn)right now, there's a shortage of truck d...
1        (cnn)working in a factory or warehouse can me...
2        (cnn)in a hong kong warehouse, a swarm of aut...
3       new york (cnn business)for many years, the wor...
4       the european union formally approved on tuesda...
                              ...                        
4071     (reuters)the australian open's covid-19 vacci...
4072     (cnn)four golfers scheduled to play in the so...
4073     (cnn)long-time international olympic committe...
4074    a norwegian technology company has found a way...
4075    this story was excerpted from the november 23 ...
Name: Article text, Length: 4076, dtype: object

##Removing news source

In [ ]:
#Removing news source prefix from articles
for a in range(len(articleText)):
  articleText[a] = articleText[a].replace(articleText[a], re.sub("\(.*?\)", "", articleText[a]))

In [ ]:
articleText

0        right now, there's a shortage of truck driver...
1        working in a factory or warehouse can mean do...
2        in a hong kong warehouse, a swarm of autonomo...
3       new york for many years, the world's most popu...
4       the european union formally approved on tuesda...
                              ...                        
4071     the australian open's covid-19 vaccine mandat...
4072     four golfers scheduled to play in the south a...
4073     long-time international olympic committee  me...
4074    a norwegian technology company has found a way...
4075    this story was excerpted from the november 23 ...
Name: Article text, Length: 4076, dtype: object

In [ ]:
articleText[0]

' right now, there\'s a shortage of truck drivers in the us and worldwide, exacerbated by the e-commerce boom brought on by the pandemic. one solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one. among them is san diego-based tusimple.founded in 2015, tusimple has completed about 2 million miles of road tests with its 70 prototype trucks across the us, china and europe. although these are simply commercially available trucks retrofitted with its technology, tusimple has deals in place with two of the world\'s largest truck manufacturers -- navistar in the us and traton, volkswagen\'s trucking business, in europe -- to design and build fully autonomous models, which it hopes to launch by 2024. photos: the yara birkeland is what its builders call the world\'s first zero-emission, autonomous cargo ship. the ship is scheduled to make its first journey between two norwegian towns before the end of the year. click through to see more

##Removing symbols

In [ ]:
#for a in range(len(articleText)):
  #articleText[a] = articleText[a].replace(articleText[a], re.sub("[^a-zA-Z0-9_\s]", "", articleText[a]))

In [ ]:
articleText

0        right now, there's a shortage of truck driver...
1        working in a factory or warehouse can mean do...
2        in a hong kong warehouse, a swarm of autonomo...
3       new york for many years, the world's most popu...
4       the european union formally approved on tuesda...
                              ...                        
4071     the australian open's covid-19 vaccine mandat...
4072     four golfers scheduled to play in the south a...
4073     long-time international olympic committee  me...
4074    a norwegian technology company has found a way...
4075    this story was excerpted from the november 23 ...
Name: Article text, Length: 4076, dtype: object

#Text Preprocessing

Stemming and Stopwords removal

In [ ]:
def text_preprocessing(sentences: list):
    """
    Pre processing text to remove unnecessary words.
    """
    # print('Preprocessing text')

    stop_words = set(stopwords.words('english'))

    clean_words = None
    for sent in sentences:
        words = word_tokenize(sent)
        words = [ps.stem(word.lower()) for word in words if word.isalnum()]
        clean_words = [word for word in words if word not in stop_words]

    return clean_words


#Process

In [ ]:
def create_tf_matrix(sentences: list) -> dict:
    """
    Here document refers to a sentence.
    TF(t) = (Number of times the term t appears in a document) / (Total number of terms in the document)
    """
    # print('Creating tf matrix.')

    tf_matrix = {}

    for sentence in sentences:
        tf_table = {}

        clean_words = text_preprocessing([sentence])
        words_count = len(word_tokenize(sentence))

        # Determining frequency of words in the sentence
        word_freq = {}
        for word in clean_words:
            word_freq[word] = (word_freq[word] + 1) if word in word_freq else 1

        # Calculating relative tf of the words in the sentence
        for word, count in word_freq.items():
            tf_table[word] = count / words_count

        tf_matrix[sentence[:15]] = tf_table

    return tf_matrix

In [ ]:
def create_idf_matrix(sentences: list) -> dict:
    """
    Inverse Document Frequency.
    IDF(t) = log_e(Total number of documents / Number of documents with term t in it)
    """
    # print('Creating idf matrix.')

    idf_matrix = {}
    documents_count = len(sentences)
    sentence_word_table = {}

    # Getting words in the sentence
    for sentence in sentences:
        clean_words = text_preprocessing([sentence])
        sentence_word_table[sentence[:15]] = clean_words

    # Determining word count table with the count of sentences which contains the word.
    word_in_docs = {}
    for sent, words in sentence_word_table.items():
        for word in words:
            word_in_docs[word] = (word_in_docs[word] + 1) if word in word_in_docs else 1

    # Determining idf of the words in the sentence.
    for sent, words in sentence_word_table.items():
        idf_table = {}
        for word in words:
            idf_table[word] = math.log10(documents_count / float(word_in_docs[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [ ]:
def create_tf_idf_matrix(tf_matrix, idf_matrix) -> dict:
    """
    Create a tf-idf matrix which is multiplication of tf * idf individual words
    """
    # print('Calculating tf-idf of sentences.')

    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):
        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(), f_table2.items()):
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [ ]:
def create_sentence_score_table(tf_idf_matrix) -> dict:
    """
    Determining average score of words of the sentence with its words tf-idf value.
    """
    # print('Creating sentence score table.')

    sentence_value = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0
        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        smoothing = 1
        sentence_value[sent] = (total_score_per_sentence + smoothing) / (count_words_in_sentence + smoothing)

    return sentence_value

In [ ]:
def find_average_score(sentence_value):
    """
    Calculate average value of a sentence form the sentence score table.
    """
    # print('Finding average score')

    sum = 0
    for val in sentence_value:
        sum += sentence_value[val]

    average = sum / len(sentence_value)

    return average

In [ ]:
def generate_summary(sentences, sentence_value, threshold):
    """
    Generate a sentence for sentence score greater than average.
    """
    # print('Generating summary')

    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentence_value and sentence_value[sentence[:15]] >= threshold:
            summary += sentence + " "
            sentence_count += 1

    return summary

In [ ]:
text = articleText[0] #Taking a sample article from the corpus

sentences = sent_tokenize(text) #Tokenizing the article into sentences to be used as documents for TF-IDF
print('Sentences', sentences)

tf_matrix = create_tf_matrix(sentences) #Finding the TF matrix of the article
print('TF matrix', tf_matrix)

idf_matrix = create_idf_matrix(sentences) #Finding the IDF matrix of the article
print('IDF matrix',idf_matrix)

tf_idf_matrix = create_tf_idf_matrix(tf_matrix, idf_matrix) #Finding the TF-IDF matrix of the article
print('TF-IDF matrix', tf_idf_matrix)
# print('First document tfidf',tf_idf_matrix[list(tf_idf_matrix.keys())[0]])

sentence_value = create_sentence_score_table(tf_idf_matrix) #Calculating the TF-IDF score of each sentences
print('Sentence Scores', sentence_value)

threshold = find_average_score(sentence_value) #Finding the threshold based on average score of the sentences
print('Threshold', threshold)

summary = generate_summary(sentences, sentence_value, threshold) #Generate the summary

print('\nOriginal document\n',text,end='\n'*2)
print('Summary\n', summary)

print()
print(f'Original {len(sent_tokenize(text))} sentences, Summarized {len(sent_tokenize(summary))} sentences')

Sentences [" right now, there's a shortage of truck drivers in the us and worldwide, exacerbated by the e-commerce boom brought on by the pandemic.", 'one solution to the problem is autonomous trucks, and several companies are in a race to be the first to launch one.', 'among them is san diego-based tusimple.founded in 2015, tusimple has completed about 2 million miles of road tests with its 70 prototype trucks across the us, china and europe.', "although these are simply commercially available trucks retrofitted with its technology, tusimple has deals in place with two of the world's largest truck manufacturers -- navistar in the us and traton, volkswagen's trucking business, in europe -- to design and build fully autonomous models, which it hopes to launch by 2024. photos: the yara birkeland is what its builders call the world's first zero-emission, autonomous cargo ship.", 'the ship is scheduled to make its first journey between two norwegian towns before the end of the year.', "cli

# Creating new CSV

Contains all the article text and summarized version with it's word and sentence count

In [ ]:
summarized = []
articleSentLength = []
articleWordLength = []
summSentLength = []
summWordLength = []

for text in articleText :
  sentences = sent_tokenize(text)

  articleSentLength.append(len(sentences))
  articleWordLength.append(len(word_tokenize(text)))

  tf_matrix = create_tf_matrix(sentences)

  idf_matrix = create_idf_matrix(sentences)

  tf_idf_matrix = create_tf_idf_matrix(tf_matrix, idf_matrix)

  sentence_value = create_sentence_score_table(tf_idf_matrix)

  threshold = find_average_score(sentence_value)

  summary = generate_summary(sentences, sentence_value, threshold)

  summarized.append(summary)
  summSentLength.append(len(sent_tokenize(summary)))
  summWordLength.append(len(word_tokenize(summary)))

data = {
    "Original Article" : articleText,
    "Original Sentence Count" : articleSentLength,
    "Original Word Count" : articleWordLength,
    "Summary" : summarized,
    "Summary Sentence Count" : summSentLength,
    "Summary Word Count" : summWordLength,
  }

df2 = pd.DataFrame(data)

df2.to_csv('/content/drive/MyDrive/csv_coll/CNN_Articels_clean/Articels_Summarized.csv')